## 1. Define Problem

* For example we solve a Sudoku
* we read a file from a data directory

In [9]:
from docplex.mp.model import Model
from docplex.mp.model_reader import ModelReader

# Read an instance of a model and variables
mdl = ModelReader.read(pathname='sudoqu.lp')

print(mdl.export_to_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: sudoqu

Minimize
 obj:
Subject To
 fa(1,1,1): x(1,1,1) = 1
 fa(1,1,2): x(1,1,2) = 0
 fa(1,1,3): x(1,1,3) = 0
 fa(1,1,4): x(1,1,4) = 0
 fa(1,1,5): x(1,1,5) = 0
 fa(1,1,6): x(1,1,6) = 0
 fa(1,1,7): x(1,1,7) = 0
 fa(1,1,8): x(1,1,8) = 0
 fa(1,1,9): x(1,1,9) = 0
 fa(1,7,1): x(1,7,1) = 0
 fa(1,7,2): x(1,7,2) = 0
 fa(1,7,3): x(1,7,3) = 0
 fa(1,7,4): x(1,7,4) = 0
 fa(1,7,5): x(1,7,5) = 0
 fa(1,7,6): x(1,7,6) = 0
 fa(1,7,7): x(1,7,7) = 1
 fa(1,7,8): x(1,7,8) = 0
 fa(1,7,9): x(1,7,9) = 0
 fa(2,2,1): x(2,2,1) = 0
 fa(2,2,2): x(2,2,2) = 1
 fa(2,2,3): x(2,2,3) = 0
 fa(2,2,4): x(2,2,4) = 0
 fa(2,2,5): x(2,2,5) = 0
 fa(2,2,6): x(2,2,6) = 0
 fa(2,2,7): x(2,2,7) = 0
 fa(2,2,8): x(2,2,8) = 0
 fa(2,2,9): x(2,2,9) = 0
 fa(3,1,1): x(3,1,1) = 0
 fa(3,1,2): x(3,1,2) = 0
 fa(3,1,3): x(3,1,3) = 0
 fa(3,1,4): x(3,1,4) = 0
 fa(3,1,5): x(3,1,5) = 0
 fa(3,1,6): x(3,1,6) = 1
 fa(3,1,7): x(3,1,7) = 0
 fa(3,1,8): x(3,1,8) = 0
 fa(3,1,9): 

## 2. Compile problem to an executable form on the chosen backend.

* Transpile to QUBO (Quadratic Unconstrained Binary Optimization) problem
* Move all conditions to the objective term via qiskit converters.


In [10]:
from qiskit.optimization import QuadraticProgram
from qiskit.optimization.converters import QuadraticProgramToIsing, InequalityToEquality, IntegerToBinary, LinearEqualityToPenalty

# define operators and converters
qp = QuadraticProgram()
ineq2eq = InequalityToEquality()
lineq2penalty = LinearEqualityToPenalty()

# import DOCplex model and apply converters
qp.from_docplex(mdl)
qp_eq = ineq2eq.convert(qp)
qubo = lineq2penalty.convert(qp_eq)

print(qubo.export_as_lp_string())

# Use Eigenvector based solvers
from qiskit import BasicAer
from qiskit.optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer
from qiskit.aqua.algorithms import QAOA, VQE, ExactEigensolver, NumPyMinimumEigensolver

# prepare Eigenvector based solvers
vqe_mes = VQE(quantum_instance=BasicAer.get_backend('statevector_simulator'))
qaoa_mes = QAOA(quantum_instance=BasicAer.get_backend('statevector_simulator'))
exact_mes = NumPyMinimumEigensolver()
vqe = MinimumEigenOptimizer(vqe_mes)   # using VQE
qaoa = MinimumEigenOptimizer(qaoa_mes)   # using QAOA
exact = MinimumEigenOptimizer(exact_mes)  # using the exact classical numpy minimum eigen solver

# Example: GAS `GroverOptimizer`
from qiskit.optimization.algorithms import GroverOptimizer

# prepare GAS solver
gas_mes = BasicAer.get_backend('statevector_simulator')
grover_optimizer = GroverOptimizer(6, num_iterations=10, quantum_instance=gas_mes)


\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: sudoqu

Minimize
 obj: - 10 x(1,1,1) - 8 x(1,1,2) - 8 x(1,1,3) - 8 x(1,1,4) - 8 x(1,1,5)
      - 8 x(1,1,6) - 8 x(1,1,7) - 8 x(1,1,8) - 8 x(1,1,9) - 8 x(1,7,1)
      - 8 x(1,7,2) - 8 x(1,7,3) - 8 x(1,7,4) - 8 x(1,7,5) - 8 x(1,7,6)
      - 10 x(1,7,7) - 8 x(1,7,8) - 8 x(1,7,9) - 8 x(2,2,1) - 10 x(2,2,2)
      - 8 x(2,2,3) - 8 x(2,2,4) - 8 x(2,2,5) - 8 x(2,2,6) - 8 x(2,2,7)
      - 8 x(2,2,8) - 8 x(2,2,9) - 8 x(3,1,1) - 8 x(3,1,2) - 8 x(3,1,3)
      - 8 x(3,1,4) - 8 x(3,1,5) - 10 x(3,1,6) - 8 x(3,1,7) - 8 x(3,1,8)
      - 8 x(3,1,9) - 8 x(3,4,1) - 8 x(3,4,2) - 10 x(3,4,3) - 8 x(3,4,4)
      - 8 x(3,4,5) - 8 x(3,4,6) - 8 x(3,4,7) - 8 x(3,4,8) - 8 x(3,4,9)
      - 8 x(3,5,1) - 8 x(3,5,2) - 8 x(3,5,3) - 8 x(3,5,4) - 8 x(3,5,5)
      - 8 x(3,5,6) - 8 x(3,5,7) - 10 x(3,5,8) - 8 x(3,5,9) - 8 x(4,3,1)
      - 8 x(4,3,2) - 8 x(4,3,3) - 8 x(4,3,4) - 8 x(4,3,5) - 8 x(4,3,6)
      - 8 x(4,3,7) - 10 x(4,3,8) - 8 x(4,3,9) 

## 3. Execute on the Quantum Computer backend

* simulated by CPU in this notebook for faster turnaround
* for real QC execution see below in BACKUP section

In [15]:
# solve and print
exact_result = exact.solve(qubo)
print("exact:", exact_result)

# TODO: use extended stabilizer

# TODO: compare to direct "mdl" solution
#exact_result_mdl = exact.solve(mdl)
#print("exact:", exact_result_mdl)

#qaoa_result = qaoa.solve(qubo)
#print("qaoa: ", qaoa_result)
#vqe_result = vqe.solve(qubo)
#print("vqe:  ",vqe_result)

ValueError: Maximum allowed size exceeded

In [12]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright